In [1]:
import numpy as np
import pandas as pd
import librosa
import time
import matplotlib.pyplot as plt
import ESC3
import multiprocessing

import tensorflow as tf
from tensorflow.keras.utils import to_categorical, plot_model

In [2]:
def PiczakNet50(input_shape):
    
    X_input = tf.keras.Input(input_shape)
    
    # First convolution block
    model = tf.keras.layers.Conv2D(80, kernel_size=(57, 6), strides=1, padding='same', name='conv0')(X_input)
    model = tf.keras.layers.Activation('relu')(model)
    model = tf.keras.layers.MaxPool2D(pool_size=(4, 3), strides=(1, 3), padding='same')(model)
    model = tf.keras.layers.Dropout(0.5)(model)
    
    # Second convolution block
    model = tf.keras.layers.Conv2D(80, kernel_size=(1, 3), strides=1, padding='same', name='conv1')(model)
    model = tf.keras.layers.Activation('relu')(model)
    model = tf.keras.layers.MaxPool2D(pool_size=(1, 3), strides=(1, 3), padding='same')(model)
    
    # Flatten
    model = tf.keras.layers.Flatten()(model)
    
    # First fully-connected block
    model = tf.keras.layers.Dense(5000, activation='relu', name='fc0')(model)
    model = tf.keras.layers.Dropout(0.5)(model)
    
    # Second fully-connected block
    model = tf.keras.layers.Dense(5000, activation='relu', name='fc1')(model)
    model = tf.keras.layers.Dropout(0.5)(model)
    
    # Output layer
    model = tf.keras.layers.Dense(50, activation=None, name='out')(model)
    
    # Create model
    model = tf.keras.Model(inputs = X_input, outputs = model, name='PiczakNet50')
    
    return model


def CreateTrainingSet50(data, label, name='', batch_size=32):
    
    # Shuffle the folds
    rnd_indices = np.arange(0, len(data))
    rnd_indices = np.random.shuffle(rnd_indices)
    
    data = data[rnd_indices].reshape((len(data), 60, 41, 3))
    label = label[rnd_indices].reshape((len(label), 50))
    
    
    data = data.astype(np.float32)
    label = label.astype(np.float32)
    
    '''
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0,
                                                              height_shift_range=0,
                                                              horizontal_flip=False,
                                                              vertical_flip=False) 
    
    # Shuffle all elements at every iteration
    training_dataset = datagen.flow(data, label, batch_size=batch_size, shuffle=True)
    '''
    
    training_dataset = tf.data.Dataset.from_tensor_slices((data, label))
    
    # Shuffle all elements at every iteration
    training_dataset = training_dataset.shuffle(len(training_dataset))
    
    # Define batch_size and prefetch size
    training_dataset = training_dataset.batch(batch_size=batch_size).prefetch(buffer_size=1)
    
    return training_dataset


def CreateValidationSet(data, label, name='', batch_size=32):
    
    data = data.astype(dtype=np.float32)
    label = label.astype(dtype=np.float32)
    
    # Create and cache training
    validation_dataset = tf.data.Dataset.from_tensor_slices((data, label))
    
    # Cache dataset
    #validation_dataset = validation_dataset.cache(name)
    
    # Define batch_size and prefetch size
    validation_dataset = validation_dataset.batch(batch_size=batch_size).prefetch(buffer_size=1)
    
    return validation_dataset

In [3]:
batch_size=64

train_d, train_l, val_d, val_l, test_d, test_l = ESC3.Load_Segments('ESC50', 1)

training_dataset = CreateTrainingSet50(train_d, train_l, name=f'train_F', batch_size=batch_size)
validation_dataset = CreateValidationSet(train_d, train_l, name=f'train_F', batch_size=batch_size)
    
# Initialize the network
net = PiczakNet50([60, 41, 3])
loss_f = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
net.compile(optimizer=opt, loss=loss_f, metrics=["accuracy"])

In [ ]:
max_epochs=50
epoch_loss, epoch_acc, epoch_vl, epoch_va = ESC3.train(net, max_epochs, training_dataset, validation_dataset, batch_size, verbose=True)

Epoch  1: 	 t-loss: 3.924480 	 t-acc: 0.018750 	 v-loss: 3.913088 	 v-acc: 0.021006 	 time: 10.971
Epoch  2: 	 t-loss: 3.922179 	 t-acc: 0.018750 	 v-loss: 3.911727 	 v-acc: 0.021006 	 time: 9.255
Epoch  3: 	 t-loss: 3.917253 	 t-acc: 0.025000 	 v-loss: 3.910533 	 v-acc: 0.023231 	 time: 9.293


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x000001DF0B1E9E58>
Traceback (most recent call last):
  File "C:\Users\Ax09\Anaconda3\envs\tf_gpu_hda\lib\site-packages\tensorflow\python\data\ops\iterator_ops.py", line 537, in __del__
    handle=self._handle, deleter=self._deleter)
  File "C:\Users\Ax09\Anaconda3\envs\tf_gpu_hda\lib\site-packages\tensorflow\python\ops\gen_dataset_ops.py", line 1279, in delete_iterator
    tld.op_callbacks, handle, deleter)
KeyboardInterrupt: 


Epoch  4: 	 t-loss: 3.926134 	 t-acc: 0.021875 	 v-loss: 3.909605 	 v-acc: 0.022403 	 time: 9.197
Epoch  5: 	 t-loss: 3.908404 	 t-acc: 0.021875 	 v-loss: 3.908722 	 v-acc: 0.022351 	 time: 9.167
Epoch  6: 	 t-loss: 3.910438 	 t-acc: 0.015625 	 v-loss: 3.907962 	 v-acc: 0.021730 	 time: 9.147
Epoch  7: 	 t-loss: 3.921241 	 t-acc: 0.028125 	 v-loss: 3.907517 	 v-acc: 0.024265 	 time: 9.060
Epoch  8: 	 t-loss: 3.906324 	 t-acc: 0.018750 	 v-loss: 3.906901 	 v-acc: 0.023231 	 time: 9.065
Epoch  9: 	 t-loss: 3.911426 	 t-acc: 0.028125 	 v-loss: 3.906878 	 v-acc: 0.022351 	 time: 9.090
Epoch 10: 	 t-loss: 3.908517 	 t-acc: 0.031250 	 v-loss: 3.906219 	 v-acc: 0.022351 	 time: 9.004
Epoch 11: 	 t-loss: 3.905114 	 t-acc: 0.015625 	 v-loss: 3.905577 	 v-acc: 0.022351 	 time: 9.013
Epoch 12: 	 t-loss: 3.915838 	 t-acc: 0.006250 	 v-loss: 3.904896 	 v-acc: 0.022351 	 time: 9.143
Epoch 13: 	 t-loss: 3.902696 	 t-acc: 0.012500 	 v-loss: 3.903943 	 v-acc: 0.013297 	 time: 9.105
Epoch 14: 	 t-loss: 